Option chains
=======

In [193]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=19,readonly=True)

intc = Stock('INTC', 'SMART', 'USD')
ib.qualifyContracts(intc)

[Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS')]

In [194]:
ib.reqMarketDataType(4)
[ticker] = ib.reqTickers(intc)
intcValue = ticker.marketPrice()
intcValue

49.36

In [195]:
chains = ib.reqSecDefOptParams(
        intc.symbol, '', intc.secType, intc.conId)
#util.df(chains)
chain = next(c for c in chains if  c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='270639', tradingClass='INTC', multiplier='100', expirations=['20200925', '20201002', '20201009', '20201016', '20201023', '20201030', '20201120', '20201218', '20210115', '20210219', '20210319', '20210416', '20210618', '20220121', '20230120'], strikes=[23.0, 25.0, 27.5, 28.0, 30.0, 32.5, 33.0, 35.0, 37.5, 38.0, 40.0, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0, 46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5, 50.0, 50.5, 51.0, 51.5, 52.0, 52.5, 53.0, 53.5, 54.0, 54.5, 55.0, 55.5, 56.0, 56.5, 57.0, 57.5, 58.0, 58.5, 59.0, 59.5, 60.0, 62.5, 65.0, 67.5, 70.0, 72.5, 75.0, 77.5, 80.0, 85.0, 90.0, 95.0, 100.0])

In [196]:
strikes = [strike for strike in chain.strikes
        if intcValue * 0.8 < strike < intcValue * 1.2]
expirations = sorted(exp for exp in chain.expirations)[:2]
rights = ['P', 'C']
#rights = ['P']

contracts = [Option('INTC', expiration, strike, right, 'SMART', tradingClass='INTC')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

Error 200, reqId 36: No security definition has been found for the request, contract: Option(symbol='INTC', lastTradeDateOrContractMonth='20200925', strike=56.5, right='P', exchange='SMART', tradingClass='INTC')
Error 200, reqId 72: No security definition has been found for the request, contract: Option(symbol='INTC', lastTradeDateOrContractMonth='20201002', strike=56.5, right='P', exchange='SMART', tradingClass='INTC')
Error 200, reqId 108: No security definition has been found for the request, contract: Option(symbol='INTC', lastTradeDateOrContractMonth='20200925', strike=56.5, right='C', exchange='SMART', tradingClass='INTC')
Error 200, reqId 144: No security definition has been found for the request, contract: Option(symbol='INTC', lastTradeDateOrContractMonth='20201002', strike=56.5, right='C', exchange='SMART', tradingClass='INTC')
Unknown contract: Option(symbol='INTC', lastTradeDateOrContractMonth='20200925', strike=56.5, right='P', exchange='SMART', tradingClass='INTC')
Unknow

140

In [203]:
from ibtypes import OptionData

ib.reqMarketDataType(2)
tickers = ib.reqTickers(*contracts)

options = []
for t in tickers:
    #print(t)
    options.append(OptionData(t))

df=util.df(options, ['conId',
        'symbol', 'lastTradeDateOrContractMonth', 'strike', 'right', 'marketPrice', 'optionYield', 'timeToExpiration', 'spread', 'bid', 'ask', 'impliedVol', 'delta', 'gamma', 'vega','theta'])
df.index=df['conId']
df


,conId,symbol,lastTradeDateOrContractMonth,strike,right,timeToExpiration,bid,ask,marketPrice,spread,optionYield,impliedVol,delta,gamma,vega,theta
conId,,,,,,,,,,,,,,,,
438743641,438743641,INTC,20200925,40.0,P,4,-1.0,-1.0,0.00,-0.0,0.000000,None,None,None,None,None
438474051,438474051,INTC,20200925,42.0,P,4,-1.0,-1.0,0.00,-0.0,0.000000,None,None,None,None,None
438474043,438474043,INTC,20200925,42.5,P,4,-1.0,-1.0,0.00,-0.0,0.000000,None,None,None,None,None
438474066,438474066,INTC,20200925,43.0,P,4,-1.0,-1.0,0.00,-0.0,0.000000,None,None,None,None,None
438474057,438474057,INTC,20200925,43.5,P,4,-1.0,-1.0,0.01,-0.0,0.000202,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443506229,443506229,INTC,20201002,57.0,C,11,-1.0,-1.0,0.02,-0.0,0.000405,None,None,None,None,None
443506219,443506219,INTC,20201002,57.5,C,11,-1.0,-1.0,0.01,-0.0,0.000202,None,None,None,None,None
443506242,443506242,INTC,20201002,58.0,C,11,-1.0,-1.0,0.01,-0.0,0.000202,None,None,None,None,None


In [199]:
expirations[0]

'20200925'

In [200]:
currentWeekPut = df[(df['right'] == 'P') & (
        df['lastTradeDateOrContractMonth'] == expirations[0])]
currentWeekPut.loc[(abs(abs(currentWeekPut.delta)-0.2)).sort_values().index].head(2)

TypeError: bad operand type for abs(): 'NoneType'

In [178]:
df1=df[(df['right']=='P') & (df['lastTradeDateOrContractMonth']==expirations[0])]
df1.loc[(abs(abs(df1.delta)-0.2)).sort_values().index].head(2)

,conId,symbol,lastTradeDateOrContractMonth,strike,right,timeToExpiration,bid,ask,bidSize,askSize,lastSize,volume,marketPrice,spread,optionYield,impliedVol,delta,gamma,vega,theta
conId,,,,,,,,,,,,,,,,,,,,
438474139,438474139,INTC,20200925,48.5,P,4,0.24,0.27,11.0,38,NaN,NaN,0.255,0.111111,0.005112,0.269849,-0.167158,0.169618,0.014409,-0.042078
438474180,438474180,INTC,20200925,49.0,P,4,0.34,0.40,91.0,37,NaN,NaN,0.370,0.150000,0.007417,0.260735,-0.260880,0.228428,0.017005,-0.052897


In [152]:
(abs(abs(df[(df['right']=='C') & (df['lastTradeDateOrContractMonth']=='20201002')].delta)-0.28)).sort_values().index

Int64Index([439458922, 439458929, 439458945, 439458939, 439458902, 439458970,
            439458958, 439458985, 439458912, 439458979, 439459005, 439458999,
            439459013, 443506229, 443506219, 443506242, 443506239, 443506253,
            439458887, 439458893, 439458864, 439458879, 439458849, 439458859,
            439458834, 439458840, 439458819, 439458825, 439458801, 439458804,
            439458780, 439458789, 439458746, 439458774, 439719419],
           dtype='int64', name='conId')

In [ ]:
df1.loc

In [142]:
(chain.expirations)[:2]

['20200925', '20201002']

In [138]:
df

,conId,symbol,lastTradeDateOrContractMonth,strike,right,timeToExpiration,bid,ask,marketPrice,spread,optionYield,impliedVol,delta,gamma,vega
conId,,,,,,,,,,,,,,,
438743641,438743641,INTC,20200925,40.0,P,4,NaN,0.12,NaN,NaN,NaN,0.640276,-0.000693,0.000680,0.000148
438474051,438474051,INTC,20200925,42.0,P,4,NaN,0.13,NaN,NaN,NaN,0.537178,-0.001530,0.001679,0.000282
438474043,438474043,INTC,20200925,42.5,P,4,NaN,0.13,NaN,NaN,NaN,0.519117,-0.002084,0.002312,0.000475
438474066,438474066,INTC,20200925,43.0,P,4,NaN,0.03,NaN,NaN,NaN,0.493525,-0.002709,0.003072,0.000508
438474057,438474057,INTC,20200925,43.5,P,4,0.02,0.08,0.050,0.75,0.001002,0.473984,-0.003732,0.004282,0.000684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443506229,443506229,INTC,20201002,57.0,C,11,-1.00,0.16,0.020,7.25,0.000401,0.363666,0.020297,0.015276,0.004092
443506219,443506219,INTC,20201002,57.5,C,11,NaN,0.15,0.010,NaN,0.000200,0.369362,0.015585,0.012019,0.004110
443506242,443506242,INTC,20201002,58.0,C,11,0.03,0.10,0.065,0.70,0.001303,0.387010,0.014407,0.010748,0.004060


In [108]:
df.loc[(df.delta).sort_values().index]

,conId,symbol,lastTradeDateOrContractMonth,strike,right,timeToExpiration,bid,ask,marketPrice,spread,optionYield,impliedVol,delta,gamma,vega
conId,,,,,,,,,,,,,,,
438474197,438474197,INTC,20200925,50.5,P,4,1.02,1.09,1.055,0.064220,0.021149,0.249446,-0.666680,0.265598,0.019571
439459179,439459179,INTC,20201002,50.5,P,11,1.20,1.55,1.375,0.225806,0.027563,0.274542,-0.589784,0.160869,0.033990
442258010,442258010,INTC,20201009,50.5,P,18,1.44,1.83,1.635,0.213115,0.032775,0.279028,-0.566831,0.128981,0.044070
438474211,438474211,INTC,20200925,50.0,P,4,0.73,0.83,0.780,0.120482,0.015636,0.250107,-0.527490,0.289543,0.021954
439459194,439459194,INTC,20201002,50.0,P,11,0.95,1.26,1.105,0.246032,0.022151,0.272178,-0.509138,0.166486,0.035197
442258022,442258022,INTC,20201009,50.0,P,18,1.23,1.52,1.375,0.190789,0.027563,0.276514,-0.502535,0.131616,0.043853
442257971,442257971,INTC,20201009,49.5,P,18,0.96,1.28,1.120,0.250000,0.022452,0.277094,-0.437190,0.129494,0.043627
439459154,439459154,INTC,20201002,49.5,P,11,0.75,1.04,0.895,0.278846,0.017941,0.274200,-0.426726,0.162555,0.035016
438474166,438474166,INTC,20200925,49.5,P,4,0.51,0.57,0.540,0.105263,0.010825,0.251843,-0.384897,0.276159,0.020984


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   symbol                        36 non-null     object 
 1   lastTradeDateOrContractMonth  36 non-null     object 
 2   strike                        36 non-null     float64
 3   right                         36 non-null     object 
 4   timeToExpiration              36 non-null     int64  
 5   bid                           36 non-null     float64
 6   ask                           36 non-null     float64
 7   marketPrice                   36 non-null     float64
 8   spread                        36 non-null     float64
 9   optionYield                   36 non-null     float64
 10  impliedVol                    36 non-null     float64
 11  delta                         36 non-null     float64
 12  gamma                         36 non-null     float64
 13  vega   

In [79]:
df.head()

,symbol,lastTradeDateOrContractMonth,strike,right,timeToExpiration,bid,ask,marketPrice,spread,optionYield,impliedVol,delta,gamma,vega
0,INTC,20200925,48.0,P,4,0.17,0.20,0.185,0.150000,0.003709,0.284028,-0.106104,0.117543,0.010135
1,INTC,20200925,48.5,P,4,0.24,0.27,0.255,0.111111,0.005112,0.269849,-0.168347,0.169534,0.014479
2,INTC,20200925,49.0,P,4,0.34,0.40,0.370,0.150000,0.007417,0.260735,-0.261796,0.227783,0.017239
3,INTC,20200925,49.5,P,4,0.51,0.57,0.540,0.105263,0.010825,0.251843,-0.384897,0.276159,0.020984
4,INTC,20200925,50.0,P,4,0.73,0.83,0.780,0.120482,0.015636,0.250107,-0.527490,0.289543,0.021954


In [52]:
ib.disconnect()